In [0]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import when,col
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col

#Chargement du fichier
spark
sc = spark.sparkContext
df = spark.read.json('s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json')
display(df.select("data.*").orderBy(df["data.price"].cast("float").desc()).limit(20))

appid,categories,ccu,developer,discount,genre,header_image,initialprice,languages,name,negative,owners,platforms,positive,price,publisher,release_date,required_age,short_description,tags,type,website
1200520,"List(Multi-player, Single-player, Co-op, LAN Co-op)",0,Fury Games,0,Action,https://cdn.akamai.steamstatic.com/steam/apps/1200520/header.jpg?t=1637367369,99900,English,Ascent Free-Roaming VR Experience,0,"0 .. 20,000","List(false, false, true)",6,99900,Fury Games,2019/12/27,0,"A premium highly-immersive VR experience developed for haptic feedback via motion floors, body suits and climate control. Go it alone or play with up to four friends in LAN co-op to destroy the infestation at the FuriCorp Research facility.","List(null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 83, null, null, null, null, null, null, null, null, null, null, null, null, 8, null, null, 10, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 10, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 9, null, null, null, null, null, 7, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 10, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 5, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 6, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 16, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 10, null)",game,http://www.furygames.net
253670,List(Single-player),0,Aartform,0,Animation & Modeling,https://cdn.akamai.steamstatic.com/steam/apps/253670/header.jpg?t=1596786459,29990,English,Aartform Curvy 3D 3.0,13,"0 .. 20,000","List(false, false, true)",31,29990,Aartform,2013/11/12,0,"Curvy 3D 3.0 is an innovative sculpting program for Windows, ideal for first time 3D artists. Drawing is Fast: Curvy brings new speed to 3D modelling by letting you draw shapes straight into 3D. Sculpting is Easy: Shapes are easy to work with using Curvy's artist friendly tools.","List(null, null, null, null, null, null, null, null, null, null, null, null, null, n

Databricks visualization. Run in Databricks to view.

La plupart des jeux sont-ils plutôt disponibles sur Windows/Mac/Linux ?

Ils sont disponibles surtout sur: Windows.

In [0]:
# Sélectionner les données pertinentes à partir du DataFrame
data = df.select("data.*")

# Calculer la répartition des jeux par plateforme
platform_counts = (
  data.agg(
    sum(col("platforms.Windows").cast("long")).alias("windows_count"),
    sum(col("platforms.Mac").cast("long")).alias("mac_count"),
    sum(col("platforms.Linux").cast("long")).alias("linux_count")
  )
)

# Afficher la répartition des jeux par plateforme
display(platform_counts)

windows_count,mac_count,linux_count
55676,12770,8458


Databricks visualization. Run in Databricks to view.

Certains genres ont-ils tendance à être disponibles préférentiellement sur certaines plateformes ?

Le genre Adventure, Sports, Massively Multiplayer etc sont disponible sur Windows.


In [0]:
from pyspark.sql.functions import expr, sum, explode, split

# Sélectionner les données pertinentes à partir du DataFrame
data = df.select("data.*")
data_exploded = data.withColumn("genre", explode(split(col("genre"), ", ")))
# Réaliser une analyse croisée entre les genres et les plateformes
cross_table = data_exploded.selectExpr("genre", "CAST(platforms.linux AS int) AS linux", "CAST(platforms.mac AS int) AS mac", "CAST(platforms.windows AS int) AS windows")
aggregated_cross_table = cross_table.groupBy("genre").agg(sum("linux").alias("total_linux"), sum("mac").alias("total_mac"), sum("windows").alias("total_windows"))

# Afficher le résultat de l'analyse croisée agrégée
display(aggregated_cross_table)


genre,total_linux,total_mac,total_windows
Education,19,56,317
Massively Multiplayer,164,270,1459
Sexual Content,7,13,54
Adventure,3302,5039,21427
Sports,287,506,2665
Accounting,0,4,16
Audio Production,7,41,193
Video Production,6,29,247
Animation & Modeling,38,74,322
Racing,304,424,2154


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.